# Accelerate PyTorch Inference using ONNXRuntime

<div style="text-align: right; font-style: italic; text-decoration: underline;"> 

[View the runnable example on GitHub](https://github.com/intel-analytics/BigDL/tree/main/python/nano/tutorial/notebook/inference/pytorch/accelerate_pytorch_inference_onnx.ipynb) 

</div> 

You can use ``Trainer.trace(accelerator='onnxruntime')`` API to enable the ONNXRuntime acceleration for PyTorch inference. It only takes a few lines.

To apply ONNXRuntime acceleration, the following dependencies and environment variables need to be set first:

In [ ]:
# for BigDL-Nano
!pip install --pre --upgrade bigdl-nano[pytorch] # install the nightly-bulit version
!source bigdl-nano-init # set environment variables for acceleration based on your current hardware

# for ONNXRuntime acceleration
!pip install onnx onnxruntime 

Let's take an [ResNet-18 model](https://pytorch.org/vision/main/models/generated/torchvision.models.resnet18.html) pretrained on ImageNet dataset as an example. First, we load the model:

In [ ]:
from torchvision.models import resnet18

model_ft = resnet18(pretrained=True)

Then we set it in evaluation mode:

In [ ]:
model_ft.eval()

To enable ONNXRuntime acceleration for your PyTorch inference pipeline, **the only change you need to made is to import BigDL-Nano Trainer, and trace your PyTorch model to convert it into an ONNXRuntime accelerated module for inference**:

In [ ]:
import torch
from bigdl.nano.pytorch import Trainer

ort_model = Trainer.trace(model_ft,
                          accelerator="onnxruntime",
                          input_sample=torch.rand(1, 3, 224, 224))

<div class="alert alert-info">

**Note**  

`input_sample` is the parameter for ONNXRuntime accelerator to know the **shape** of the model input. So both the batch size and the specific values are not important to `input_sample`. If we want our test dataset to consist of images with $224 \times 224$ pixels, we could use `torch.rand(1, 3, 224, 224)` for `input_sample` here. 

`input_sample` is not required if you have used an instance of ``Trainer`` to fit your model before, or the model is a `LightningModule` with any dataloader attached.

Please refer to [API documentation](https://bigdl.readthedocs.io/en/latest/doc/PythonAPI/Nano/pytorch.html#bigdl.nano.pytorch.Trainer.trace) for more information on `Trainer.trace`.

</div>

You could then do the normal inference steps with the model optimized by ONNXRuntime:

In [ ]:
x = torch.rand(2, 3, 224, 224)
# use the optimized model here
y_hat = ort_model(x)
predictions = y_hat.argmax(dim=1)
print(predictions)

<div class="alert alert-info">

**Related Readings**

- [How to install BigDL-Nano](https://bigdl.readthedocs.io/en/latest/doc/Nano/Overview/nano.html#install)
- [How to install BigDL-Nano in Google Colab](https://bigdl.readthedocs.io/en/latest/doc/Nano/Howto/install_in_colab.html)
- How to load/save optimized models

</div>